In [ ]:
!pip install google-search-results geocoder
!pip install -q -U google-generativeai gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
import geocoder
from serpapi import GoogleSearch
import requests
from datetime import datetime
import json
import google.generativeai as genai
from gtts import gTTS
import IPython.display as ipd

GOOGLE_API_KEY = "api-key"
genai.configure(api_key=GOOGLE_API_KEY)
SERPAPI_KEY = "api-key"

location = f"Atlanta"
print("Detected Location:", location)


def get_weather(location):
    params = {
        "q": f"{location} weather today",
        "hl": "en",
        "api_key": SERPAPI_KEY
    }
    results = GoogleSearch(params).get_dict()

    answer = results.get("answer_box", {})

    if not answer:
        return "Weather data not found"

    weather_report = {
        "temperature": answer.get("temperature"),
        "unit": answer.get("unit"),
        "condition": answer.get("weather"),
        "precipitation": answer.get("precipitation"),
        "humidity": answer.get("humidity"),
        "wind": answer.get("wind")
    }

    return weather_report if weather_report else {"error": "Forecast not found"}


def get_air_quality(location):
    try:
        WAQI_API_TOKEN = "api-key"
        city = location.split(",")[0]
        waqi_url = f"https://api.waqi.info/feed/{city}/?token={WAQI_API_TOKEN}"
        resp = requests.get(waqi_url).json()

        if resp and resp.get("status") == "ok":
            data = resp.get("data", {})
            if data:
                aqi_value = data.get("aqi")
                if aqi_value is not None:
                    return {"aqi": aqi_value, "dominant_pollutant": data.get("dominentpol"), "source": "WAQI API"}

    except Exception as e:
        print(f"Error fetching from WAQI API: {e}")
        pass

    try:
        params = {
            "q": f"{location} AQI",
            "hl": "en",
            "api_key": SERPAPI_KEY
        }
        results = GoogleSearch(params).get_dict()

        if "organic_results" in results:
            for res in results["organic_results"]:
                snippet = res.get("snippet", "")
                if "AQI" in snippet or "Air Quality" in snippet:
                    return snippet
    except Exception as e:
        print(f"Error fetching from SerpAPI: {e}")
        pass


    return "AQI data not available"


def get_tech_news():
    params = {
        "engine": "google_news",
        "q": "Today's top AI tech company news",
        "hl": "en",
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    news_list = []
    for article in results.get("news_results", [])[:5]:
        news_list.append(f"- {article['title']} ({article['link']})")

    return "\n".join(news_list) if news_list else "No news found."

gemini_model = genai.GenerativeModel('gemini-2.5-flash')

def summarize_with_gemini(briefing_json):
    today = datetime.now().strftime("%B %d, %Y")

    content = (f"""
        Good morning, John! Let's get you caught up with your friendly morning briefing.
        Today is {today}.
        Here is the data:
        {briefing_json}
        Write a friendly spoken morning briefing that includes weather, air quality, and top AI tech news.
        Please write the morning briefing in the following style:
        - Start with a cheerful greeting.
        - Mention the current date explicitly.
        - Summarize the weather first (condition, temperature, rain %, humidity, wind).
          * If precipitation chance is ≥ 40%, add: "Take an umbrella with you as it might rain."
        - Summarize air quality clearly (AQI value + category + main pollutant). Suggest some actions to be taken.
        - Then summarize the top 3–4 AI Tech news headlines as bullet points, written like a human radio announcer.
        - End with a positive closing like: "That's your morning brief for today! Have a fantastic day!"

        Keep it conversational, structured, and polished like a spoken briefing.
        """
    )
    response = gemini_model.generate_content(content)
    return response.text

def text_to_speech(summary_text, filename="morning_briefing.mp3"):
    tts = gTTS(summary_text)
    tts.save(filename)
    return ipd.Audio(filename)

📍 Detected Location: Atlanta


In [ ]:
class MorningBriefingAgent:
    def __init__(self):
        self.memory = {}

    def run(self):
        print("Morning Briefing Agent started...")

        # Step 1: Detect location
        print(f"Location detected: {location}")

        # Step 2: Fetch data using tools
        print("Fetching weather...")

        self.memory["weather"] = get_weather(location)

        print("Fetching air quality...")
        self.memory["aqi"] = get_air_quality(location)

        print("Fetching AI tech news...")
        self.memory["tech_news"] = get_tech_news()

        # Step 3: Generate structured output
        briefing = {
            "location": location,
            "weather": self.memory["weather"],
            "air_quality": self.memory["aqi"],
            "tech_news": self.memory["tech_news"]
        }

        briefing_text = summarize_with_gemini(briefing)

        print("Spoken Morning Briefing:\n", briefing_text)
        #print("\nData JSON:\n", json.dumps(briefing, indent=2))

        return briefing_text

agent = MorningBriefingAgent()
briefing = agent.run()

audio = text_to_speech(briefing)
display(audio)


🤖 Morning Briefing Agent started...
📍 Location detected: Atlanta
🔎 Fetching weather...
🔎 Fetching air quality...
🔎 Fetching AI tech news...
🗣 Spoken Morning Briefing:
 Good morning, Pavan! Rise and shine, let's get you caught up with your friendly morning briefing.

Today is September 03, 2025.

First, let's check on the weather in Atlanta. You can expect partly cloudy skies with a comfortable 68 degrees Fahrenheit. Humidity is at 78%, and there's a gentle breeze of 3 mph. With no precipitation expected, it looks like a lovely day ahead!

The air quality is looking great today, Pavan! The AQI is at a very healthy 32, which falls into the 'Good' category, with PM2.5 being the dominant pollutant. This is perfect for enjoying any outdoor activities you have planned!

Now, for your top AI tech news headlines:

*   First up, it seems AI stocks are facing a 'show me' moment, with Nvidia apparently not quite wowing Wall Street as expected.
*   Next, a significant warning from California's Att

Wake Word Detection (“Hey AB”)

In [ ]:
import pvporcupine
import pyaudio

porcupine = pvporcupine.create(keywords=["hey ab"])
# Keeps listening for wake word

Speech Recognition (what you said after wake word)

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()
with sr.Microphone() as source:
    audio = r.listen(source)
    command = r.recognize_google(audio)

Response Generation (AB gives briefing)

In [ ]:
summary = summarize_with_gemini(briefing_json, location="New York")
tts = gTTS(summary)
tts.save("ab_briefing.mp3")
os.system("afplay ab_briefing.mp3")   # Mac

Good morning, New York! Let's get you caught up with your friendly morning briefing.

Looks like we're starting off with some lovely weather. It's a **clear 66 degrees Fahrenheit**, with a light breeze at 3 mph. Humidity is a bit high at 76%, but with 0% chance of precipitation, you can **definitely leave that umbrella at home today!**

For your air quality, the AQI is looking great this morning at a very healthy **24**, with PM2.5 being the dominant pollutant. That's well within the 'Good' range, so enjoy that fresh air!

Now, let's dive into some of the top headlines from the tech world, particularly focusing on AI:

*   First up, AI stocks are facing a 'show me' moment, with even giants like **Nvidia recently failing to completely wow Wall Street**, as investors look for concrete results.
*   On the regulatory side, **California's Attorney General Bonta has issued a stern warning to AI companies**, stating that if their technology harms children, they will be held accountable. A clear message about ethical responsibility.
*   And in the global AI race, tech stocks saw a bit of a wobble as **China's new DeepSeek platform sparked renewed U.S. concerns** about the pace and competition in artificial intelligence.
*   Plus, for those planning ahead, McKinsey has just released their **2025 Technology Trends Outlook**, offering a glimpse into future innovations.

That's your morning brief for today! Have a fantastic day, New York!


Good morning, New York, USA! Rise and shine!

It's September 03, 2025, and here's your friendly morning briefing to get your day started.

Let's begin with your local weather. You're waking up to a beautiful, clear day with a comfortable 66 degrees Fahrenheit. Humidity is at 76%, and you can expect a very light breeze at just 3 miles per hour. Best of all, there's a 0% chance of precipitation today, so no need for an umbrella!

Moving on to our air quality report, it's looking excellent! The Air Quality Index is a low 24, which falls firmly into the 'Good' category. The dominant pollutant is PM2.5, but at this level, it means you can comfortably enjoy all your outdoor activities today without any worries.

Now, let's dive into the world of AI tech, with some of the headlines making waves this morning:

*   First up, it seems AI stocks are facing a 'show me' moment, with reports questioning why Nvidia failed to impress Wall Street.
*   Next, we have important news from California, where Attorney General Bonta is sending a strong warning to AI companies: if you harm children, you will be held accountable.
*   And finally, the competitive spirit in the tech world heats up as tech stocks are reportedly falling, sparked by China's DeepSeek, raising U.S. worries about the ongoing AI race.

That's your morning brief for today! Have a fantastic day, New York, USA!